<img src="teacher.jpg">

# $ \Delta $ $上市公司日報酬率標準差_i$ = $a_0$ + $a_1$ * (上市公司現股當沖比重平均)

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.stats.api as sms
import statsmodels.api as sm
import statsmodels.formula.api as smf
import seaborn as sns
import warnings
import datetime as datetime
from datetime import timedelta
warnings.filterwarnings("ignore")
%matplotlib inline

from sklearn.linear_model import LinearRegression
from scipy import stats

### 選取2013/06/13 - 2019/06/20 每間台灣上市公司的日報酬率、週報酬率、月報酬率以及現股當沖比重的資料

In [2]:
day_trade_data = pd.read_csv("20130613_20190620_data_utf8.csv")

In [3]:
day_trade_data.head(n=8)

,證券代碼,Data Field,2013/6/13,2013/6/14,2013/6/17,2013/6/18,2013/6/19,2013/6/20,2013/6/21,2013/6/24,...,2019/6/6,2019/6/10,2019/6/11,2019/6/12,2019/6/13,2019/6/14,2019/6/17,2019/6/18,2019/6/19,2019/6/20
0,1101 台泥,日報酬率 %,-1.9685,0.1339,1.0695,-0.6614,0.0000,-0.6658,-5.7641,-1.5647,...,0.3488,1.6222,0.3421,-0.5682,0.2286,0.2281,1.7065,1.1186,0.2212,0.2208
1,1101 台泥,週報酬率 %,-0.4001,-1.1889,-1.6905,-1.4436,-1.4436,-0.1339,-6.0161,-8.4657,...,0.5828,1.0368,2.0881,1.7441,1.6222,1.8540,1.9385,2.7274,3.5429,3.5349
2,1101 台泥,月報酬率 %,-3.3637,-4.3480,-4.3039,-4.9369,-4.9369,-4.9683,-11.0128,-11.6220,...,2.0093,3.1763,3.5293,2.9410,4.4047,5.7761,5.7988,6.9823,7.2190,7.2020
3,1101 台泥,現股當沖比重,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,...,11.5700,7.1600,5.2100,9.7500,8.4100,2.5800,15.1100,7.1200,9.9700,2.3800
4,1102 亞泥,日報酬率 %,-0.8174,-0.2747,0.2755,0.2747,-0.5479,-1.3774,-2.0950,0.2853,...,0.4381,0.1091,1.8519,0.0000,-1.2834,0.4334,0.8630,0.5348,0.8511,0.5274
5,1102 亞泥,週報酬率 %,-0.5464,-2.6810,-2.5434,-0.5449,-1.0898,-1.6483,-3.4435,-3.4341,...,1.6629,0.8791,3.2009,2.4097,0.6544,1.0907,1.8520,0.5349,1.3905,3.2504
6,1102 亞泥,月報酬率 %,-2.8039,-3.7137,-3.9578,-3.6940,-4.2217,-5.1656,-7.5199,-5.8902,...,10.2161,10.0718,12.1102,12.1102,11.4733,13.0487,6.8572,7.4286,8.3430,8.2956
7,1102 亞泥,現股當沖比重,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,...,13.7300,10.9600,11.2500,10.7400,15.2700,12.5200,16.8800,8.4800,15.2700,21.9700


In [4]:
day_trade_data.shape

(3796, 1481)

In [8]:
# df for dataframe, str for the split date
def find_date_split(df,str):
    tmp_arr = np.array(list(df))
    split_result = np.where(tmp_arr==str)[0]
    return split_result

def day_trade(str1, str2, str3):
    d1 = find_date_split(day_trade_data, str1)[0]
    d2 = find_date_split(day_trade_data, str2)[0]
    d3 = find_date_split(day_trade_data, str3)[0]

    day_trade_data_half_year_before = day_trade_data.iloc[:,d1:d2]
    day_trade_data_half_year_after = day_trade_data.iloc[:,d2:(d3+1)]

    # 計算每間公司的日報酬率/週報酬率/年報酬率 標準差
    day_trade_data_half_year_before['company_std_before'] = day_trade_data_half_year_before.std(axis=1)
    day_trade_data_half_year_after['company_std_after'] = day_trade_data_half_year_after.std(axis=1)

    # 計算每間公司的現股當沖比重平均
    day_trade_data_half_year_before['company_day_mean_before'] = day_trade_data_half_year_before.mean(axis=1)
    day_trade_data_half_year_after['company_mean_after'] = day_trade_data_half_year_after.mean(axis=1)

    # 計算x和y
    x = day_trade_data_half_year_after.iloc[3::4]['company_mean_after']
    y_day_return = day_trade_data_half_year_after.iloc[0::4]['company_std_after'] - day_trade_data_half_year_before.iloc[0::4]['company_std_before']
    y_week_return = day_trade_data_half_year_after.iloc[1::4]['company_std_after'] - day_trade_data_half_year_before.iloc[1::4]['company_std_before']
    y_month_return = day_trade_data_half_year_after.iloc[2::4]['company_std_after'] - day_trade_data_half_year_before.iloc[2::4]['company_std_before']
    
    result = [x, y_day_return, y_week_return, y_month_return]
    return result
    

def OLS_test(x,y):
    y = list(y)
    x = sm.add_constant(x) # a_0
    model = sm.OLS(y,x).fit()
    print(model.summary())

### 2014/1/6 開放單向當沖，前後各抓半年(2013/7/5 和 2015/7/6)，對日報酬/週報酬/月報酬的影響

In [15]:
result = day_trade('2013/7/5','2014/1/6','2015/7/6')
print(OLS_test(result[0],result[1]))
print(OLS_test(result[0],result[2]))
print(OLS_test(result[0],result[3]))

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.002
Model:                            OLS   Adj. R-squared:                  0.001
Method:                 Least Squares   F-statistic:                     2.244
Date:                Mon, 24 Jun 2019   Prob (F-statistic):              0.134
Time:                        01:23:00   Log-Likelihood:                -1194.3
No. Observations:                 949   AIC:                             2393.
Df Residuals:                     947   BIC:                             2402.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
const                  0.1072      0

### 2014/6/30 開放雙向當沖，前後各抓半年(2014/1/27 和 2014/12/30)，對日報酬/週報酬/月報酬的影響

In [17]:
result = day_trade('2014/1/27','2014/6/30','2014/12/30')
print(OLS_test(result[0],result[1]))
print(OLS_test(result[0],result[2]))
print(OLS_test(result[0],result[3]))

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.007
Model:                            OLS   Adj. R-squared:                  0.006
Method:                 Least Squares   F-statistic:                     6.415
Date:                Mon, 24 Jun 2019   Prob (F-statistic):             0.0115
Time:                        01:24:08   Log-Likelihood:                -1098.5
No. Observations:                 949   AIC:                             2201.
Df Residuals:                     947   BIC:                             2211.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
const                  0.0481      0

### 2014/6/30 開放雙向當沖，前後各抓一年(2013/6/30 和 2015/6/30)，對日報酬/週報酬/月報酬的影響

In [23]:
result = day_trade('2013/6/28','2014/6/30','2015/6/30')
print(OLS_test(result[0],result[1]))
print(OLS_test(result[0],result[2]))
print(OLS_test(result[0],result[3]))

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                 -0.001
Method:                 Least Squares   F-statistic:                   0.04552
Date:                Mon, 24 Jun 2019   Prob (F-statistic):              0.831
Time:                        01:27:27   Log-Likelihood:                -1051.5
No. Observations:                 949   AIC:                             2107.
Df Residuals:                     947   BIC:                             2117.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
const                  0.0578      0

### 2017/4/28 調降當沖稅率，前後各抓一年(2016/4/28 和 2018/4/28)，對日報酬/週報酬/月報酬的影響

In [25]:
result = day_trade('2016/4/28','2017/4/28','2018/4/27')
print(OLS_test(result[0],result[1]))
print(OLS_test(result[0],result[2]))
print(OLS_test(result[0],result[3]))

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.023
Model:                            OLS   Adj. R-squared:                  0.022
Method:                 Least Squares   F-statistic:                     22.68
Date:                Mon, 24 Jun 2019   Prob (F-statistic):           2.22e-06
Time:                        01:29:13   Log-Likelihood:                -1435.5
No. Observations:                 949   AIC:                             2875.
Df Residuals:                     947   BIC:                             2885.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
const                 -0.1232      0

### 2017/4/28 調降當沖稅率，前後各抓兩年(2015/4/28 和 2019/4/28)，對日報酬/週報酬/月報酬的影響

In [29]:
result = day_trade('2015/4/28','2017/4/28','2019/4/26')
print(OLS_test(result[0],result[1]))
print(OLS_test(result[0],result[2]))
print(OLS_test(result[0],result[3]))

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.031
Model:                            OLS   Adj. R-squared:                  0.030
Method:                 Least Squares   F-statistic:                     30.62
Date:                Mon, 24 Jun 2019   Prob (F-statistic):           4.06e-08
Time:                        01:30:34   Log-Likelihood:                -1284.0
No. Observations:                 949   AIC:                             2572.
Df Residuals:                     947   BIC:                             2582.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
const                 -0.2985      0